In [1]:
import re
import pandas as pd
import plotly.express as px

from glob import glob

In [2]:
pt_time = re.compile(r"Done epoch (\d): Time (\d+\.\d+)")
pt_node = re.compile(r"node(\d+)")
pt_up = re.compile(r"up(\d+)")

In [3]:
logs = sorted(glob("./logs/**/torch.log"))
rows = []

for log in logs:
    with open(log, "r") as f:
        lines = f.readlines()

    time_per_epoch = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
    for line in lines:
        sh_time = pt_time.search(line)
        if sh_time:
            time_per_epoch[int(sh_time.group(1))] = float(sh_time.group(2))

    up = int(pt_up.search(log).group(1))
    if up == 4:
        resolution = "1980 x 1480"
    elif up == 8:
        resolution = "3960 x 2960"

    node_num = int(pt_node.search(log).group(1))
    if node_num == 4:
        gpus = "A100 * 4"
    if node_num == 5:
        gpus = "H100 * 8"
    elif node_num == 7:
        gpus = "L40 * 4"
    elif node_num == 8:
        gpus = "A40 * 4"
    elif node_num == 9:
        gpus = "L4 * 4"

    rows.append(
        {
            "gpus": gpus,
            "storage": log.split("-")[3],
            "resolution": resolution,
            "epoch_1": round(time_per_epoch[1], 2),
            "epoch_2": round(time_per_epoch[2], 2),
            "epoch_3": round(time_per_epoch[3], 2),
            "epoch_4": round(time_per_epoch[4], 2),
            "epoch_5": round(time_per_epoch[5], 2),
        }
    )

In [8]:
df = pd.DataFrame(rows)
df.sort_values(by=["resolution"], ascending=[True], inplace=True)
df.reset_index(drop=True, inplace=True)
df["mean"] = round(df.iloc[:, [3, 4, 5, 6, 7]].mean(axis=1), 3)
df["standard_deviation"] = round(df.iloc[:, [3, 4, 5, 6, 7]].std(axis=1), 3)
df.to_csv("data_gds.csv", index=False)
df

,gpus,storage,resolution,epoch_1,epoch_2,epoch_3,epoch_4,epoch_5,mean,standard_deviation
0,H100 * 8,gpfs,1980 x 1480,12.00,9.48,9.62,9.47,9.36,9.986,1.130
1,H100 * 8,ontap,1980 x 1480,25.99,22.93,25.88,26.69,25.44,25.386,1.444
2,H100 * 8,gpfs,3960 x 2960,12.01,10.76,11.45,11.93,19.34,13.098,3.525
3,H100 * 8,ontap,3960 x 2960,85.74,79.46,76.26,73.54,74.35,77.870,4.955
